# Cohort Building

**Example use case:** 

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" alt="alt_text" align="left"
	width="150" height="150" />
Julia is an oncologist that specializes in female reproductive health. As part of her research, she is interested in  using existing data on uterine cancers. If possible, she would like to see multiple datatypes (gross imaging, genomic data, proteomic data, histology) that come from the same patient, so she can look for shared phenotypes to test for their potential as early diagnostics. Julia heard that the Cancer Data Aggregator has made it easy to search across multiple datasets created by NCI, and so has decided to start her search there.



## Getting Started

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---

Before Julia does any work, she needs to import these functions cdapython.
She'll also need to import [pandas](https://pandas.pydata.org/) to get nice dataframes.
Finally, she tells cdapython to report it's version so she can be sure she's using the one she means to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
import pandas as pd 
print(cdapython.__version__)

2022.7.13

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in five main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. An individual who participates in 3 studies will have 3 researchsubject IDs</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.
</div>


## Finding Search Terms

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
   
   Accordingly, to see what search fields are available, Julia starts by using the command `columns`:

In [2]:
columns().to_list()

['File.id',
 'File.identifier.system',
 'File.identifier.value',
 'File.label',
 'File.data_category',
 'File.data_type',
 'File.file_format',
 'File.associated_project',
 'File.drs_uri',
 'File.byte_size',
 'File.checksum',
 'File.data_modality',
 'File.imaging_modality',
 'File.dbgap_accession_number',
 'File.imaging_series',
 'id',
 'identifier.system',
 'identifier.value',
 'species',
 'sex',
 'race',
 'ethnicity',
 'days_to_birth',
 'subject_associated_project',
 'vital_status',
 'days_to_death',
 'cause_of_death',
 'ResearchSubject.id',
 'ResearchSubject.identifier.system',
 'ResearchSubject.identifier.value',
 'ResearchSubject.member_of_research_project',
 'ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubject.Diagnosis.identifier.value',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubje

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
   
There are a lot of columns in the CDA data, but Julia is most interested in diagnosis data, so she filters the list to only those:

In [3]:
columns().to_list(filters="diagnosis")

['ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubject.Diagnosis.identifier.value',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubject.Diagnosis.morphology',
 'ResearchSubject.Diagnosis.stage',
 'ResearchSubject.Diagnosis.grade',
 'ResearchSubject.Diagnosis.method_of_diagnosis',
 'ResearchSubject.Diagnosis.Treatment.id',
 'ResearchSubject.Diagnosis.Treatment.identifier.system',
 'ResearchSubject.Diagnosis.Treatment.identifier.value',
 'ResearchSubject.Diagnosis.Treatment.treatment_type',
 'ResearchSubject.Diagnosis.Treatment.treatment_outcome',
 'ResearchSubject.Diagnosis.Treatment.days_to_treatment_start',
 'ResearchSubject.Diagnosis.Treatment.days_to_treatment_end',
 'ResearchSubject.Diagnosis.Treatment.therapeutic_agent',
 'ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site',
 'Re

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">

To search the CDA, a user also needs to know what search terms are available. Each column will contain a huge amount of data, so retrieving all of the rows would be overwhelming. Instead, the CDA has a `unique_terms()` function that will return all of the unique values that populate the requested column. Like `columns`, `unique_terms` defaults to giving us an overview of the results, and can be filtered.
    
</div>

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Since Julia is interested specifically in uterine cancers, she uses the `unique_terms` function to see what data is available for 'ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site' and 'ResearchSubject.primary_diagnosis_site' to see if 'uterine' appears:

In [4]:
unique_terms("ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site").to_list()

['Brain',
 'Cervix',
 'Head - Face Or Neck, Nos',
 'Lymph Node(s) Paraaortic',
 'Other',
 'Pelvis',
 'Spine',
 'Unknown']

In [5]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list()

['Abdomen',
 'Abdomen, Mediastinum',
 'Abdomen, Pelvis',
 'Adrenal Glands',
 'Adrenal gland',
 'Anus and anal canal',
 'Base of tongue',
 'Bile Duct',
 'Bladder',
 'Bones, joints and articular cartilage of limbs',
 'Bones, joints and articular cartilage of other and unspecified sites',
 'Brain',
 'Breast',
 'Bronchus and lung',
 'Cervix',
 'Cervix uteri',
 'Chest',
 'Chest-Abdomen-Pelvis, Leg, TSpine',
 'Colon',
 'Connective, subcutaneous and other soft tissues',
 'Corpus uteri',
 'Ear',
 'Esophagus',
 'Extremities',
 'Eye and adnexa',
 'Floor of mouth',
 'Gallbladder',
 'Gum',
 'Head',
 'Head and Neck',
 'Head-Neck',
 'Heart, mediastinum, and pleura',
 'Hematopoietic and reticuloendothelial systems',
 'Hypopharynx',
 'Intraocular',
 'Kidney',
 'Larynx',
 'Lip',
 'Liver',
 'Liver and intrahepatic bile ducts',
 'Lung',
 'Lung Phantom',
 'Lymph nodes',
 'Marrow, Blood',
 'Meninges',
 'Mesothelium',
 'Nasal cavity and middle ear',
 'Nasopharynx',
 'Not Reported',
 'Oropharynx',
 'Other an

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA makes multiple datasets searchable from a common interface, but does not harmonize the data. This means that researchers should review all the terms in a column, and not just choose the first one that fits, as there may be other similar terms available as well.
    
</div>

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Julia sees that "treatment_anatomic_site" does not have 'Uterine', but does have 'Cervix'. She also notes that both 'Uterus' and 'Uterus, NOS' are listed in the "primary_diagnosis_site" results. As she was initially looking for "uterine", Julia decides to expand her search a bit to account for variable naming schemes. So, she runs a fuzzy match filter on the "ResearchSubject.primary_diagnosis_site" for 'uter' as that should cover all variants:

In [6]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list(filters="uter")

['Cervix uteri', 'Corpus uteri', 'Uterus', 'Uterus, NOS']

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Just to be sure, Julia also searches for any other instances of "cervix":

In [7]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list(filters="cerv")

['Cervix', 'Cervix uteri']

## Building a Query

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
With all her likely terms found, Julia begins to create a search that will get data for all of her terms. She does this by writing a series of `Q` statements that define what rows should be returned from each column. For the "treatment_anatomic_site", only one term is of interest, so she uses the `=` operator to get only exact matches:

In [8]:
Tsite = Q('ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site = "Cervix"')

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
However, for "primary_diagnosis_site", Julia has several terms she wants to search with. Luckily, `Q` also can run fuzzy searches. It can also search more than one term at a time, so Julia writes one big `Q` statement to grab everything that is either 'uter' or 'cerv':

In [9]:
Dsite = Q('ResearchSubject.primary_diagnosis_site = "%uter%" OR ResearchSubject.primary_diagnosis_site = "%cerv%"')

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Finally, Julia adds her two queries together into one large one:

In [10]:
ALLDATA = Tsite.OR(Dsite)

## Looking at Summary Data

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Now that Julia has a query, she can use it to look for data in any of the CDA endpoints. She starts by getting an overall summary of what data is available using `count`:

In [11]:
ALLDATA.count.run()

Getting results from database

Total execution time: 0 min 10.255 sec 10255 ms

specimen_count : 40793

treatment_count : 3049

diagnosis_count : 3685

researchsubject_count : 4869

subject_count : 3742

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
It seems there's a lot of data that might work for Julias study! Since she is interested in the beginings of cancer, she decides to start by looking at  the researchsubject information, since that is where most of the diagnosis information is. She again gets a summary using `count`:

In [12]:
ALLDATA.researchsubject.count.run()

Getting results from database

Total execution time: 0 min 7.371 sec 7371 ms

total : 4869

files : 324646

system,count
GDC,3591
PDC,104
IDC,1174
primary_diagnosis_condition,count
Complex Mixed and Stromal Neoplasms,320
Adenomas and Adenocarcinomas,1672
Squamous Cell Neoplasms,609
Myomatous Neoplasms,188
Uterine Corpus Endometrial Carcinoma,104
"Cystic, Mucinous and Serous Neoplasms",487


## Refining Queries

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Browsing the primary_diagnosis_condition data, Julia notices that there are a large number of research subjects that are Adenomas and Adenocarcinomas. Since Julia wants to look for common phenotypes in early cancers, she decides it might be easier to exclude the endocrine related data, as they might have different mechanisms. So she adds a new filter to her query:

In [13]:
Noadeno = Q('ResearchSubject.primary_diagnosis_condition != "Adenomas and Adenocarcinomas"')

NoAdenoData = ALLDATA.AND(Noadeno)

NoAdenoData.researchsubject.count.run()

Getting results from database

Total execution time: 0 min 7.123 sec 7123 ms

total : 3197

files : 298263

system,count
GDC,1919
PDC,104
IDC,1174
primary_diagnosis_condition,count
Myomatous Neoplasms,188
Uterine Corpus Endometrial Carcinoma,104
"Cystic, Mucinous and Serous Neoplasms",487
Squamous Cell Neoplasms,609
Complex Mixed and Stromal Neoplasms,320
None,1175


<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
She then previews the actual metadata for researchsubject, subject, and file, to make sure that they have all the information she will need for her work. Since she's mostly interested in looking at the kinds of data available from each endpoint:

In [14]:
NoAdenoData.researchsubject.run().to_dataframe() # view the dataframe

Getting results from database

Total execution time: 0 min 3.852 sec 3852 ms

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,0ba55ef3-9381-4b8f-a98b-39a7c8c7ce40,"[{'system': 'GDC', 'value': '0ba55ef3-9381-4b8...",GENIE-MSK,Myomatous Neoplasms,"Uterus, NOS",GENIE-MSK-P-0002272
1,1cc0625d-118a-11e9-afb9-0a9c39d33490,"[{'system': 'PDC', 'value': '1cc0625d-118a-11e...",CPTAC3-Discovery,Uterine Corpus Endometrial Carcinoma,"Uterus, NOS",C3L-00771
2,24a13674-3ab6-4870-9b22-1b2042c961f1,"[{'system': 'GDC', 'value': '24a13674-3ab6-487...",GENIE-GRCC,Squamous Cell Neoplasms,Cervix uteri,GENIE-GRCC-febe36a1
3,2810e7d0-a4d5-4b07-8475-73aa66e4abed,"[{'system': 'GDC', 'value': '2810e7d0-a4d5-4b0...",GENIE-DFCI,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-DFCI-052989
4,2c7f9b9b-26be-4959-b986-3a210c24f7fb,"[{'system': 'GDC', 'value': '2c7f9b9b-26be-495...",GENIE-UHN,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-UHN-380128
...,...,...,...,...,...,...
95,010a807f-9dc0-4e14-9533-dcf478f3d947,"[{'system': 'GDC', 'value': '010a807f-9dc0-4e1...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-C5-A902
96,038cbbdd-f477-4b77-9fa0-04fcdbc06819,"[{'system': 'GDC', 'value': '038cbbdd-f477-4b7...",GENIE-MSK,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-MSK-P-0002829
97,085df27c-9bc2-425a-9a49-547d692ea028,"[{'system': 'GDC', 'value': '085df27c-9bc2-425...",TCGA-CESC,"Cystic, Mucinous and Serous Neoplasms",Cervix uteri,TCGA-C5-A3HF
98,092844dd-58a8-45d5-a2fa-1f5c55fe9cf6,"[{'system': 'GDC', 'value': '092844dd-58a8-45d...",GENIE-MSK,Myomatous Neoplasms,"Uterus, NOS",GENIE-MSK-P-0004731


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Study(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

In [15]:
NoAdenoData.subject.run().to_dataframe() # view the dataframe

Getting results from database

Total execution time: 0 min 3.971 sec 3971 ms

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,AD17519,"[{'system': 'GDC', 'value': 'AD17519'}]",homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
1,C3L-00563,"[{'system': 'GDC', 'value': 'C3L-00563'}, {'sy...",homo sapiens,female,asian,not hispanic or latino,-22777.0,"[CPTAC3-Discovery, CPTAC-3, cptac_ucec]",Alive,NaN,Not Reported
2,C3L-02449,"[{'system': 'IDC', 'value': 'C3L-02449'}]",homo sapiens,None,None,None,NaN,[cptac_ucec],None,NaN,None
3,C3N-00836,"[{'system': 'GDC', 'value': 'C3N-00836'}, {'sy...",homo sapiens,female,not reported,not reported,-27509.0,"[CPTAC3-Discovery, CPTAC-3, cptac_ucec]",Alive,NaN,Not Reported
4,C3N-00847,"[{'system': 'GDC', 'value': 'C3N-00847'}, {'sy...",homo sapiens,female,not reported,not reported,-23757.0,"[CPTAC3-Discovery, CPTAC-3, cptac_ucec]",Alive,NaN,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...
95,TCGA-AJ-A3NH,"[{'system': 'GDC', 'value': 'TCGA-AJ-A3NH'}, {...",homo sapiens,female,white,not hispanic or latino,-32871.0,"[tcga_ucec, TCGA-UCEC]",Dead,1.0,None
96,TCGA-AJ-A8CW,"[{'system': 'GDC', 'value': 'TCGA-AJ-A8CW'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-23554.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
97,TCGA-B5-A0KB,"[{'system': 'GDC', 'value': 'TCGA-B5-A0KB'}, {...",homo sapiens,female,black or african american,not reported,-22669.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
98,TCGA-B5-A11N,"[{'system': 'GDC', 'value': 'TCGA-B5-A11N'}, {...",homo sapiens,female,white,not reported,-25442.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.",STRING</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the patient. For MVP, since taxonomy vocabulary is consistent between GDC and PDC, using text. Ultimately, this will be a term returned by the vocabulary service.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

In [16]:
NoAdenoData.file.run().to_dataframe() # view the dataframe

Getting results from database

Total execution time: 0 min 21.698 sec 21698 ms

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,imaging_series,researchsubject_specimen_id,researchsubject_id,subject_id
0,011eb8a3-3af7-4864-acac-81bf342e1235,"[{'system': 'GDC', 'value': '011eb8a3-3af7-486...",e8dc552a-7a49-4352-891a-681e9182054a.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:011eb8a3-3af7-4864-acac-81bf342e...,55962,02642e1373ff9b1a46d661c39cd30e70,Genomic,None,phs000528,None,ad6e1d47-441c-4f3f-a497-4966999e397c,ba5e5f27-689b-49e4-a5f9-4007a8bcd2f0,HTMCP-03-06-02195
1,029be527-fd5c-4b34-a863-edc2bbc4b72e,"[{'system': 'GDC', 'value': '029be527-fd5c-4b3...",ecaf8912-be90-491d-803b-870a40152d71.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:029be527-fd5c-4b34-a863-edc2bbc4...,68703,94863e6c2926eda219a5a8ea45b5329f,Genomic,None,phs000528,None,9a06f2e6-1095-4388-953f-022977e02b5e,e9230570-37a0-4bb6-bf21-170e830fe9dc,HTMCP-03-06-02447
2,16c2b877-92a2-4880-8274-f938cfd9d07e,"[{'system': 'GDC', 'value': '16c2b877-92a2-488...",b31b8644-c18e-4a15-a97b-73475f56c64e.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:16c2b877-92a2-4880-8274-f938cfd9...,50383,bddc054ef0117d8760a091c8d069db70,Genomic,None,phs000528,None,b40e5621-f344-4e95-858f-55d4889c0d0e,e5e4671c-9a3f-418c-a37f-ef37f95262b1,HTMCP-03-06-02008
3,16c2b877-92a2-4880-8274-f938cfd9d07e,"[{'system': 'GDC', 'value': '16c2b877-92a2-488...",b31b8644-c18e-4a15-a97b-73475f56c64e.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:16c2b877-92a2-4880-8274-f938cfd9...,50383,bddc054ef0117d8760a091c8d069db70,Genomic,None,phs000528,None,bf5489ba-67d4-4264-9d37-b08e82c4af55,e5e4671c-9a3f-418c-a37f-ef37f95262b1,HTMCP-03-06-02008
4,191171de-dad7-4a8b-ba61-56f0e391f2c4,"[{'system': 'GDC', 'value': '191171de-dad7-4a8...",94f253aa-8001-48d2-947c-50eda73e29da.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:191171de-dad7-4a8b-ba61-56f0e391...,46865,a5f7e0f2a4b378591e1461053b71b61a,Genomic,None,phs000528,None,917aed83-30ff-4312-b1cf-f8e271a70073,59673b19-2e14-441f-9624-626ed0a3530b,HTMCP-03-06-02125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2dccf286-954f-4f30-b514-2660b3c5ed32,"[{'system': 'GDC', 'value': '2dccf286-954f-4f3...",94374ed9-b99d-40e2-ab45-3fe6aedfe539.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:2dccf286-954f-4f30-b514-2660b3c5...,101310,aae3de35fc56627d24eefa316577fd42,Genomic,None,phs000528,None,be0c8e07-d119-4da8-9b2e-0340565b864c,37941fa0-0167-4bfb-a609-29521ee854d8,HTMCP-03-06-02266
96,76296e79-20b9-4a9b-a7e5-03404c8cbd3e,"[{'system': 'GDC', 'value': '76296e79-20b9-4a9...",06a9ed2a-d3c3-4432-847b-f4cd1bac19ba.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:76296e79-20b9-4a9b-a7e5-03404c8c...,204931,2e328e3b66d9f5b9027c15eb211c0a6e,Genomic,None,phs000528,None,e39ea57e-9fb1-46e6-9235-85a412bdab68,ec6c18ae-1e62-4c7b-84b9-8b795b9da775,HTMCP-03-06-02109
97,78f6a75d-90ec-49fb-8d7a-64dd06fdcf39,"[{'system': 'GDC', 'value': '78f6a75d-90ec-49f...",TCGA-GH-A9DA-01A-11-A40H-20_RPPA_data.tsv,Proteome Profiling,Protein Expression Quantification,TSV,TCGA-CESC,drs://dg.4DFC:78f6a75d-90ec-49fb-8d7a-64dd06fd...,22086,bd651cefecfde729a799aecc7a1ff91e,Genomic,None,None,None,f936df60-3e10-4e16-865c-c28e916c595d,67195401-9099-4ca9-9790-f42bcb815289,TCGA-GH-A9DA
98,7ceead93-cd15-4ac8-8636-ce6cebdcde98,"[{'system': 'GDC', 'value': '7ceead93-cd15-4ac...",d1c774e3-31ab-48aa-8141-4e4de8c2f460.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:7ceead93-cd15-4ac8-8636-ce6cebdc...,46514,4bbd60d6ee0cb3a77aebac3ef5addabd,Genomic,None,phs000528,None,6644a


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an Activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---


## Working with Results (pagination)

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Finally, Julia wants to save these results to use for the future. Since the preview dataframes only show the first 100 results of each search, she uses the `paginator` function to get all the data from the subject and researchsubject endpoints into their own dataframes:

In [17]:
researchsubs = NoAdenoData.researchsubject.run()
rsdf = researchsubs.auto_paginator(to_df=True)

Getting results from database

Total execution time: 0 min 3.998 sec 3998 ms

Output()

In [18]:
subs = NoAdenoData.subject.run()
subsdf = subs.auto_paginator(to_df=True)

Getting results from database

Total execution time: 0 min 3.454 sec 3454 ms

Output()

In [19]:
rsdf # view the researchsubject dataframe

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,b330c651-2a5f-483a-b872-9f6f290ace31,"[{'system': 'GDC', 'value': 'b330c651-2a5f-483...",GENIE-DFCI,Complex Mixed and Stromal Neoplasms,"Uterus, NOS",GENIE-DFCI-009251
1,b5b56f2d-69f8-4299-ab0b-fbd5868d7b94,"[{'system': 'GDC', 'value': 'b5b56f2d-69f8-429...",FM-AD,Squamous Cell Neoplasms,Cervix uteri,AD11962
2,c5887926-8d20-43c1-b94e-f60ef5406f72,"[{'system': 'GDC', 'value': 'c5887926-8d20-43c...",GENIE-DFCI,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-DFCI-001837
3,c996cffb-8132-4973-987d-143274b6a081,"[{'system': 'GDC', 'value': 'c996cffb-8132-497...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-EK-A2RO
4,c99f476b-e8b1-4cdd-9164-0da450064886,"[{'system': 'GDC', 'value': 'c99f476b-e8b1-4cd...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-C5-A905
...,...,...,...,...,...,...
95,010a807f-9dc0-4e14-9533-dcf478f3d947,"[{'system': 'GDC', 'value': '010a807f-9dc0-4e1...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-C5-A902
96,038cbbdd-f477-4b77-9fa0-04fcdbc06819,"[{'system': 'GDC', 'value': '038cbbdd-f477-4b7...",GENIE-MSK,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-MSK-P-0002829
97,085df27c-9bc2-425a-9a49-547d692ea028,"[{'system': 'GDC', 'value': '085df27c-9bc2-425...",TCGA-CESC,"Cystic, Mucinous and Serous Neoplasms",Cervix uteri,TCGA-C5-A3HF
98,092844dd-58a8-45d5-a2fa-1f5c55fe9cf6,"[{'system': 'GDC', 'value': '092844dd-58a8-45d...",GENIE-MSK,Myomatous Neoplasms,"Uterus, NOS",GENIE-MSK-P-0004731


In [20]:
subsdf # view the subject dataframe

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,TCGA-5B-A90C,"[{'system': 'GDC', 'value': 'TCGA-5B-A90C'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-25518.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
1,TCGA-A5-A1OK,"[{'system': 'GDC', 'value': 'TCGA-A5-A1OK'}, {...",homo sapiens,female,white,not hispanic or latino,-23179.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
2,TCGA-B5-A1MW,"[{'system': 'GDC', 'value': 'TCGA-B5-A1MW'}, {...",homo sapiens,female,white,not reported,-20183.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
3,TCGA-C5-A1BF,"[{'system': 'GDC', 'value': 'TCGA-C5-A1BF'}, {...",homo sapiens,female,white,not reported,-16975.0,"[tcga_cesc, TCGA-CESC]",Dead,570.0,None
4,TCGA-EY-A2OQ,"[{'system': 'GDC', 'value': 'TCGA-EY-A2OQ'}, {...",homo sapiens,female,white,not hispanic or latino,-22309.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
95,TCGA-AJ-A3NH,"[{'system': 'GDC', 'value': 'TCGA-AJ-A3NH'}, {...",homo sapiens,female,white,not hispanic or latino,-32871.0,"[tcga_ucec, TCGA-UCEC]",Dead,1.0,None
96,TCGA-AJ-A8CW,"[{'system': 'GDC', 'value': 'TCGA-AJ-A8CW'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-23554.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
97,TCGA-B5-A0KB,"[{'system': 'GDC', 'value': 'TCGA-B5-A0KB'}, {...",homo sapiens,female,black or african american,not reported,-22669.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None
98,TCGA-B5-A11N,"[{'system': 'GDC', 'value': 'TCGA-B5-A11N'}, {...",homo sapiens,female,white,not reported,-25442.0,"[tcga_ucec, TCGA-UCEC]",Alive,NaN,None


## Merging Results across Endpoints

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Then Julia uses the `subject_id` and `id` fields in each result to merge them together into one big dataset. She also specifies that any columns that are in both tables should be kept and have a suffix added to their name. This will help her to check that her merge worked correctly:

In [21]:
allmetadata = pd.merge(rsdf,
                subsdf,
                left_on="subject_id",
                right_on='id',
                suffixes=("_rs", "_sub"))

allmetadata

,id_rs,identifier_rs,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id,id_sub,identifier_sub,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,b330c651-2a5f-483a-b872-9f6f290ace31,"[{'system': 'GDC', 'value': 'b330c651-2a5f-483...",GENIE-DFCI,Complex Mixed and Stromal Neoplasms,"Uterus, NOS",GENIE-DFCI-009251,GENIE-DFCI-009251,"[{'system': 'GDC', 'value': 'GENIE-DFCI-009251'}]",homo sapiens,female,white,not hispanic or latino,-24471.0,[GENIE-DFCI],Not Reported,NaN,None
1,b5b56f2d-69f8-4299-ab0b-fbd5868d7b94,"[{'system': 'GDC', 'value': 'b5b56f2d-69f8-429...",FM-AD,Squamous Cell Neoplasms,Cervix uteri,AD11962,AD11962,"[{'system': 'GDC', 'value': 'AD11962'}]",homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
2,c5887926-8d20-43c1-b94e-f60ef5406f72,"[{'system': 'GDC', 'value': 'c5887926-8d20-43c...",GENIE-DFCI,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-DFCI-001837,GENIE-DFCI-001837,"[{'system': 'GDC', 'value': 'GENIE-DFCI-001837'}]",homo sapiens,female,white,not hispanic or latino,-23741.0,[GENIE-DFCI],Not Reported,NaN,None
3,c996cffb-8132-4973-987d-143274b6a081,"[{'system': 'GDC', 'value': 'c996cffb-8132-497...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-EK-A2RO,TCGA-EK-A2RO,"[{'system': 'GDC', 'value': 'TCGA-EK-A2RO'}, {...",homo sapiens,female,american indian or alaska native,hispanic or latino,-21844.0,"[tcga_cesc, TCGA-CESC]",Alive,NaN,None
4,TCGA-EK-A2RO__tcga_cesc,"[{'system': 'IDC', 'value': 'TCGA-EK-A2RO__tcg...",tcga_cesc,None,Cervix,TCGA-EK-A2RO,TCGA-EK-A2RO,"[{'system': 'GDC', 'value': 'TCGA-EK-A2RO'}, {...",homo sapiens,female,american indian or alaska native,hispanic or latino,-21844.0,"[tcga_cesc, TCGA-CESC]",Alive,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,d6bb6e54-aaec-4e88-957d-44c1257d3cbd,"[{'system': 'GDC', 'value': 'd6bb6e54-aaec-4e8...",GENIE-DFCI,Complex Mixed and Stromal Neoplasms,"Uterus, NOS",GENIE-DFCI-004931,GENIE-DFCI-004931,"[{'system': 'GDC', 'value': 'GENIE-DFCI-004931'}]",homo sapiens,female,white,not hispanic or latino,-20819.0,[GENIE-DFCI],Not Reported,NaN,None
3193,eaadfe2a-23fb-400b-86b3-4fba6d968aa3,"[{'system': 'GDC', 'value': 'eaadfe2a-23fb-400...",TCGA-SARC,Myomatous Neoplasms,"Uterus, NOS",TCGA-IS-A3K6,TCGA-IS-A3K6,"[{'system': 'GDC', 'value': 'TCGA-IS-A3K6'}, {...",homo sapiens,female,white,not reported,-20205.0,"[tcga_sarc, TCGA-SARC]",Dead,858.0,None
3194,038cbbdd-f477-4b77-9fa0-04fcdbc06819,"[{'system': 'GDC', 'value': '038cbbdd-f477-4b7...",GENIE-MSK,"Epithelial Neoplasms, NOS",Corpus uteri,GENIE-MSK-P-0002829,GENIE-MSK-P-0002829,"[{'system': 'GDC', 'value': 'GENIE-MSK-P-00028...",homo sapiens,female,white,not hispanic or latino,-24837.0,[GENIE-MSK],Not Reported,NaN,None
3195,092844dd-58a8-45d5-a2fa-1f5c55fe9cf6,"[{'system': 'GDC', 'value': '092844dd-58a8-45d...",GENIE-MSK,Myomatous Neoplasms,"Uterus, NOS",GENIE-MSK-P-0004731,GENIE-MSK-P-0004731,"[{'system': 'GDC', 'value': 'GENIE-MSK-P-00047...",homo sapiens,female,white,not hispanic or latino,-18993.0,[GENIE-MSK],Not Reported,NaN,None


<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
`subject_id` from the research subject results seems to perfectly match the id data from the subject table, `id_sub`. Julia then checks to see that her dataframe is the right size. She had 3197 researchsubject rows, so she expects 3197 rows here as well:

In [22]:
allmetadata.count()

id_rs                          3197
identifier_rs                  3197
member_of_research_project     3197
primary_diagnosis_condition    2022
primary_diagnosis_site         3197
subject_id                     3197
id_sub                         3197
identifier_sub                 3197
species                        3197
sex                            3025
race                           3025
ethnicity                      3025
days_to_birth                  2740
subject_associated_project     3197
vital_status                   3025
days_to_death                   482
cause_of_death                  309
dtype: int64

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Satisfied with her results, Julia saves the data out to a csv so she can browse it with Excel:

In [23]:
allmetadata.to_csv("allmetadata.csv")

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
   
Julia knows from her subject count summary that there are more than 200,000 files associated with her subjects, which is likely far more than she needs. To help her decide what files she wants, Julia uses endpoint chaining to get summary information about the files that are assigned to researchsubjects for her search criteria:


In [24]:
NoAdenoData.researchsubject.file.count.run()

Getting results from database

Total execution time: 0 min 10.158 sec 10158 ms

total : 298263

system,count
IDC,264429
PDC,2560
GDC,31274
data_category,count
Imaging,264429
Biospecimen,2866
Simple Nucleotide Variation,11745
Sequencing Reads,4142
Copy Number Variation,4079
DNA Methylation,1947


<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
    
Julia decides that a good place to start would be with Slide Images. There's only 1111, so she should be able to quickly scan through them over the next few days and see if they will be useful. So she adds one more filter on her search:

In [25]:
JustSlides = Q('file.data_type = "Slide Image"')
NoadenoJustSlides = NoAdenoData.AND(JustSlides)
NoadenoJustSlides.researchsubject.file.count.run()

Getting results from database

Total execution time: 0 min 7.763 sec 7763 ms

total : 1111

system,count
GDC,1111
data_category,count
Biospecimen,1111
file_format,count
SVS,1111
data_type,count
Slide Image,1111


<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
    
Finally, Julia uses the pagenation function again to get all the slide files, and merges her metadata file with this file information. This way she will be able to review what phenotypes each slide is associated with:

In [26]:
slides = NoadenoJustSlides.researchsubject.file.run()
slidesdf = slides.auto_paginator(to_df=True)

Getting results from database

Total execution time: 0 min 7.105 sec 7105 ms

Output()

In [27]:
slidemetadata = pd.merge(slidesdf, 
                         allmetadata, 
                         left_on=("subject_id","researchsubject_id"),
                         right_on=("subject_id", "id_rs"),
                         suffixes=("_slide", "_all"))
slidemetadata

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,...,identifier_sub,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,30e840fe-49d4-4162-a278-b46ff40cc89d,"[{'system': 'GDC', 'value': '30e840fe-49d4-416...",TCGA-N8-A4PO-01Z-00-DX1.D73029FC-ADE9-4FA3-BE9...,Biospecimen,Slide Image,SVS,TCGA-UCS,drs://dg.4DFC:30e840fe-49d4-4162-a278-b46ff40c...,1567650278,c6c4d283b89bdadced0fb34d24d6652d,...,"[{'system': 'GDC', 'value': 'TCGA-N8-A4PO'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-24124.0,"[tcga_ucs, TCGA-UCS]",Alive,NaN,None
1,aea60480-458b-4c9a-9443-f5c0df54302b,"[{'system': 'GDC', 'value': 'aea60480-458b-4c9...",TCGA-N8-A4PO-01A-01-TS1.06618EE3-C5AB-4A28-A4B...,Biospecimen,Slide Image,SVS,TCGA-UCS,drs://dg.4DFC:aea60480-458b-4c9a-9443-f5c0df54...,116870159,d99004d654cf2494cde96fde434377ae,...,"[{'system': 'GDC', 'value': 'TCGA-N8-A4PO'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-24124.0,"[tcga_ucs, TCGA-UCS]",Alive,NaN,None
2,3392b8e7-2360-4757-9957-7980440db1b0,"[{'system': 'GDC', 'value': '3392b8e7-2360-475...",TCGA-AJ-A23M-01A-01-TSA.6682b6d5-0289-490f-aa1...,Biospecimen,Slide Image,SVS,TCGA-UCEC,drs://dg.4DFC:3392b8e7-2360-4757-9957-7980440d...,261402339,4f0834b59142a52f0357c849947917cb,...,"[{'system': 'GDC', 'value': 'TCGA-AJ-A23M'}, {...",homo sapiens,female,white,not reported,-22430.0,"[tcga_ucec, TCGA-UCEC]",Dead,616.0,None
3,4ac5933d-e8ba-4d5c-baf4-9b8c667a5200,"[{'system': 'GDC', 'value': '4ac5933d-e8ba-4d5...",TCGA-AJ-A23M-01Z-00-DX1.F2FD9BFC-8A07-4F62-AFF...,Biospecimen,Slide Image,SVS,TCGA-UCEC,drs://dg.4DFC:4ac5933d-e8ba-4d5c-baf4-9b8c667a...,118644520,1ba8af4dcedd8d49b92f59855a4c40c2,...,"[{'system': 'GDC', 'value': 'TCGA-AJ-A23M'}, {...",homo sapiens,female,white,not reported,-22430.0,"[tcga_ucec, TCGA-UCEC]",Dead,616.0,None
4,3a103211-35bb-43e9-9928-bc2ce5f077b3,"[{'system': 'GDC', 'value': '3a103211-35bb-43e...",TCGA-FL-A1YH-11A-01-TSA.a2a056a5-8da6-45c1-b2f...,Biospecimen,Slide Image,SVS,TCGA-UCEC,drs://dg.4DFC:3a103211-35bb-43e9-9928-bc2ce5f0...,33549799,7b3b6745f945df0976e94fe7f3fc42de,...,"[{'system': 'GDC', 'value': 'TCGA-FL-A1YH'}, {...",homo sapiens,None,None,None,NaN,"[tcga_ucec, TCGA-UCEC]",None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,bcdbf3c9-cdde-4832-a8ee-6f52d32f2585,"[{'system': 'GDC', 'value': 'bcdbf3c9-cdde-483...",TCGA-EA-A3HR-01A-01-TSA.BF26D24D-F4AE-44A7-BED...,Biospecimen,Slide Image,SVS,TCGA-CESC,drs://dg.4DFC:bcdbf3c9-cdde-4832-a8ee-6f52d32f...,358454545,f008cf148a2269fd53b89fb5a8648656,...,"[{'system': 'GDC', 'value': 'TCGA-EA-A3HR'}, {...",homo sapiens,female,white,not hispanic or latino,-20980.0,"[tcga_cesc, TCGA-CESC]",Alive,NaN,None
1107,3213e3ff-5d34-4354-8095-4acfec55a3de,"[{'system': 'GDC', 'value': '3213e3ff-5d34-435...",TCGA-EA-A3HR-01Z-00-DX1.E3ED09EE-1667-42DD-A1F...,Biospecimen,Slide Image,SVS,TCGA-CESC,drs://dg.4DFC:3213e3ff-5d34-4354-8095-4acfec55...,1667987777,09233ae2d3493c8bf80c52661c8e0e34,...,"[{'system': 'GDC', 'value': 'TCGA-EA-A3HR'}, {...",homo sapiens,female,white,not hispanic or latino,-20980.0,"[tcga_cesc, TCGA-CESC]",Alive,NaN,None
1108,25a4f1dd-9a59-4d08-8228-c2b9ca57bd2c,"[{'system': 'GDC', 'value': '25a4f1dd-9a59-4d0...",TCGA-EK-A2H0-01A-01-TSA.1F5A10E3-2C42-4623-9AD...,Biospecimen,Slide Image,SVS,TCGA-CESC,drs://dg.4DFC:25a4f1dd-9a59-4d08-8228-c2b9ca57...,119352685,e0c68507f94d34cd88d94cc022237f67,...,"[{'system': 'GDC', 'value': 'TCGA-EK-A2H0'}, {...",homo sapiens,female,white,not reported,-8918.0,"[tcga_cesc, TCGA-CESC]",Alive,NaN,None
1109,695b6c77-9d6c-45bf-9a73-465c1f2ea8a8,"[{'system': 'GDC', 'value': '695b6c77-9d6c-45b...",TCGA-FL-A1YI-11A-01-TSA.6d819055-db2e-443f-b7c...,Biospecimen,Slide Image,SVS,TCGA-UCEC,drs://dg.4DFC:695b6c77-9d6c-45bf-9a73-465c1f2e...,75844161,ae22fd4d4fbc9abacea0911baaaacd25,...,"[{'sy

In [28]:
slidemetadata.count()

id                             1111
identifier                     1111
label                          1111
data_category                  1111
data_type                      1111
file_format                    1111
associated_project             1111
drs_uri                        1111
byte_size                      1111
checksum                       1111
data_modality                  1111
imaging_modality                  0
dbgap_accession_number            0
imaging_series                    0
subject_id                     1111
researchsubject_id             1111
id_rs                          1111
identifier_rs                  1111
member_of_research_project     1111
primary_diagnosis_condition    1111
primary_diagnosis_site         1111
id_sub                         1111
identifier_sub                 1111
species                        1111
sex                            1099
race                           1099
ethnicity                      1099
days_to_birth               

<img src="https://github.com/CancerDataAggregator/readthedocs/blob/main/docs/Examples/images/Julia.png?raw=true" align="left"
	width="50" height="50" />
    
Julia saves this dataframe to a csv as well, and now she has all the information she needs to begin work on her project. She can use the `drs_id` column information to directly download the images she is interested in using a DRS resolver, or she can input the DRS IDs at a cloud workspace such as [Terra](https://terra.bio/) or the [Cancer Genomics Cloud](https://www.cancergenomicscloud.org/) to view the images online. In either case, she has all the metadata she needs to get started, and can save this notebook of her work in case she'd like to come back and modify her search.

In [29]:
slidemetadata.to_csv("slidemetadata.csv")